In [13]:
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pickle
import json
from SPARQLWrapper import SPARQLWrapper, JSON
from surprise import SVD
from surprise import Reader
from surprise import Dataset
from surprise import evaluate, print_perf
from surprise import accuracy
from surprise.prediction_algorithms.knns import *
%matplotlib inline

In [2]:
ratings = pd.read_csv( '/Users/jose/Documents/Maestria/Sistemas de recomendación/Taller 3/ml-latest/ratings.csv', sep = ',', engine = 'python' )

In [3]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,110,1.0,1425941529
1,1,147,4.5,1425942435
2,1,858,5.0,1425941523
3,1,1221,5.0,1425941546
4,1,1246,5.0,1425941556


In [4]:
ratings.count()

userId       26024289
movieId      26024289
rating       26024289
timestamp    26024289
dtype: int64

In [5]:
movies = pd.read_csv( '/Users/jose/Documents/Maestria/Sistemas de recomendación/Taller 3/ml-latest/movies.csv', sep = ',', engine = 'python' )

In [6]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [7]:
movies.count()

movieId    45843
title      45843
genres     45843
dtype: int64

In [8]:
tags = pd.read_csv( '/Users/jose/Documents/Maestria/Sistemas de recomendación/Taller 3/ml-latest/tags.csv', sep = ',', engine = 'python' )

In [9]:
tags.head()

,userId,movieId,tag,timestamp
0,1,318,narrated,1425942391
1,20,4306,Dreamworks,1459855607
2,20,89302,England,1400778834
3,20,89302,espionage,1400778836
4,20,89302,jazz,1400778841


In [10]:
tags.count()

userId       753170
movieId      753170
tag          753154
timestamp    753170
dtype: int64

In [3]:
links = pd.read_csv( '/Users/jose/Documents/Maestria/Sistemas de recomendación/Taller 3/ml-latest/links.csv', sep = ',', engine = 'python' )

In [4]:
links.head()

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


In [13]:
links.count()

movieId    45843
imdbId     45843
tmdbId     45624
dtype: int64

In [14]:
genomeScores = pd.read_csv( '/Users/jose/Documents/Maestria/Sistemas de recomendación/Taller 3/ml-latest/genome-scores.csv', sep = ',', engine = 'python' )

In [15]:
genomeScores.head()

,movieId,tagId,relevance
0,1,1,0.02475
1,1,2,0.02475
2,1,3,0.04900
3,1,4,0.07750
4,1,5,0.12450


In [16]:
genomeScores.count()

movieId      12400104
tagId        12400104
relevance    12400104
dtype: int64

In [17]:
genomeTags = pd.read_csv( '/Users/jose/Documents/Maestria/Sistemas de recomendación/Taller 3/ml-latest/genome-tags.csv', sep = ',', engine = 'python' )

In [18]:
genomeTags.head()

,tagId,tag
0,1,007
1,2,007 (series)
2,3,18th century
3,4,1920s
4,5,1930s


In [19]:
genomeTags.count()

tagId    1128
tag      1128
dtype: int64

In [20]:
tags['DateTime'] = pd.to_datetime(tags['timestamp'], unit='s')

In [21]:
tags.head()

,userId,movieId,tag,timestamp,DateTime
0,1,318,narrated,1425942391,2015-03-09 23:06:31
1,20,4306,Dreamworks,1459855607,2016-04-05 11:26:47
2,20,89302,England,1400778834,2014-05-22 17:13:54
3,20,89302,espionage,1400778836,2014-05-22 17:13:56
4,20,89302,jazz,1400778841,2014-05-22 17:14:01


In [22]:
ratings['DateTime'] = pd.to_datetime(ratings['timestamp'], unit='s')

In [23]:
ratings.head()

,userId,movieId,rating,timestamp,DateTime
0,1,110,1.0,1425941529,2015-03-09 22:52:09
1,1,147,4.5,1425942435,2015-03-09 23:07:15
2,1,858,5.0,1425941523,2015-03-09 22:52:03
3,1,1221,5.0,1425941546,2015-03-09 22:52:26
4,1,1246,5.0,1425941556,2015-03-09 22:52:36


In [24]:
date_rating = pd.DataFrame(ratings['DateTime']).drop_duplicates()

In [25]:
date_rating.describe()

,DateTime
count,20549435
unique,20549435
top,2013-04-24 11:28:21
freq,1
first,1995-01-09 11:46:44
last,2017-08-04 06:57:50


In [26]:
users_rating = pd.DataFrame(ratings['userId']).drop_duplicates()

In [27]:
users_rating.describe()

,userId
count,270896.000000
mean,135448.500000
std,78201.083599
min,1.000000
25%,67724.750000
50%,135448.500000
75%,203172.250000
max,270896.000000


In [28]:
genomes = pd.merge(genomeScores,genomeTags, on=['tagId'])

In [29]:
genomes.head()

,movieId,tagId,relevance,tag
0,1,1,0.02475,007
1,2,1,0.03750,007
2,3,1,0.03800,007
3,4,1,0.03125,007
4,5,1,0.04100,007


In [30]:
genomes.count()

movieId      12400104
tagId        12400104
relevance    12400104
tag          12400104
dtype: int64

In [31]:
pelicula = pd.DataFrame(tags[tags['movieId']==1]['tag']).drop_duplicates()
pelicula.sort_values('tag')

,tag
657468,2009 reissue in Stereoscopic 3-D
614820,3D
657469,55 movies every kid should see--Entertainment ...
736440,American Animation
355068,Animation
657470,BD-Video
706177,Best of Rotten Tomatoes: All Time
686606,Buzz Lightyear
686607,CG animation
510255,CGI


In [32]:
pelicula = movies[movies['movieId']==781]
pelicula

,movieId,title,genres
768,781,Stealing Beauty (1996),Drama


In [27]:
def get_movie_description():
    sparql = SPARQLWrapper("http://dbpedia.org/sparql")
    sparql.setReturnFormat(JSON)
    total_results = 16
    offset = 0
    out = []
 
    while total_results > 0:
            sparql.setQuery("""
            SELECT ?label ?film ?Rdate ?imdbId ?country ?director 
            WHERE { ?film rdf:type dbo:Film . 
                    OPTIONAL { ?film rdfs:label ?label }. 
                    OPTIONAL { ?film dbo:releaseDate ?Rdate }.
                    OPTIONAL { ?film dbo:imdbId ?imdbId }. 
                    OPTIONAL { ?film dbp:country ?country }. 
                    OPTIONAL { ?film dbo:director ?director }. 
            FILTER (lang(?label) = \'en\') } 
            OFFSET """ + str(offset) + """
            LIMIT 10000""")

            result = sparql.query()
            offset = offset + 10000
            
            processed_results = json.load(result.response)
            cols = processed_results['head']['vars']

            for row in processed_results['results']['bindings']:
                item = []
                for c in cols:
                    item.append(row.get(c, {}).get('value'))
                out.append(item)

            total_results=total_results-1
    resultM = pd.DataFrame(out, columns=cols)
    return resultM

In [28]:
resultDBPEDIA = get_movie_description()
resultDBPEDIA

,label,film,Rdate,imdbId,country,director
0,THX 1138,http://dbpedia.org/resource/THX_1138,1971-03-11,0066434,United States,http://dbpedia.org/resource/George_Lucas
1,The Big Fisherman,http://dbpedia.org/resource/The_Big_Fisherman,1959-08-04,0052627,United States,http://dbpedia.org/resource/Frank_Borzage
2,The Hollywood Revue of 1929,http://dbpedia.org/resource/The_Hollywood_Revu...,1929-06-20,0019993,United States,http://dbpedia.org/resource/Charles_Reisner
3,The War Game,http://dbpedia.org/resource/The_War_Game,1965-11-01,0059894,UK,http://dbpedia.org/resource/Peter_Watkins
4,Bad Company (1972 film),http://dbpedia.org/resource/Bad_Company_(1972_...,1972-10-08,0068245,United States,http://dbpedia.org/resource/Robert_Benton
5,Bad Company (1999 film),http://dbpedia.org/resource/Bad_Company_(1999_...,1999-10-20,0211504,France,http://dbpedia.org/resource/Jean-Pierre_Améris
6,Blown Away (1994 film),http://dbpedia.org/resource/Blown_Away_(1994_f...,1994-07-01,0109303,United States,http://dbpedia.org/resource/Stephen_Hopkins_(d...
7,Coup de Grâce (1969 film),http://dbpedia.org/resource/Coup_de_Grâce_(196...,1969-10-02,0183898,Argentina,http://dbpedia.org/resource/Ricardo_Becher
8,God Is My Co-Pilot (film),http://dbpedia.org/resource/God_Is_My_Co-Pilot...,1945-02-21,0037746,United States,http://dbpedia.org/resource/Robert_Florey
9,Jack of All Trades (1936 film),http://dbpedia.org/resource/Jack_of_All_Trades...,1936-12-30,0030905,United Kingdom,http://dbpedia.org/resource/Robert_Stevenson_(...


In [12]:
resultDBPEDIA.describe()

,label,film,Rdate,imdb,country,director
count,144319,144319,26180,34952,97615,108208
unique,129591,129591,15285,30676,1210,27997
top,Bridges of Sarajevo,http://dbpedia.org/resource/Bridges_of_Sarajevo,2001-12-12,0055747,United States,http://dbpedia.org/resource/D._W._Griffith
freq,78,78,18,25,40101,279


In [26]:
film_date = pd.DataFrame(resultDBPEDIA['Rdate'])
film_date2=film_date.drop_duplicates()
fd=film_date.loc[film_date['Rdate'].isnull()]
fd

,Rdate
59,None
60,None
61,None
62,None
63,None
64,None
65,None
66,None
67,None
68,None


In [35]:
resultDBPEDIA['ID'] = pd.to_numeric(resultDBPEDIA.imdbId, errors='coerce')
links['ID'] = pd.to_numeric(links.imdbId, errors='coerce')

In [36]:
links

,movieId,imdbId,tmdbId,ID
0,1,114709,862.0,114709
1,2,113497,8844.0,113497
2,3,113228,15602.0,113228
3,4,114885,31357.0,114885
4,5,113041,11862.0,113041
5,6,113277,949.0,113277
6,7,114319,11860.0,114319
7,8,112302,45325.0,112302
8,9,114576,9091.0,114576
9,10,113189,710.0,113189


In [37]:
dataOntology = pd.merge(links,resultDBPEDIA, on=['ID'])

In [38]:
dataOntology

,movieId,imdbId_x,tmdbId,ID,label,film,Rdate,imdbId_y,country,director
0,9,114576,9091.0,114576,Sudden Death (1995 film),http://dbpedia.org/resource/Sudden_Death_(1995...,None,0114576,United States,http://dbpedia.org/resource/Peter_Hyams
1,11,112346,9087.0,112346,The American President,http://dbpedia.org/resource/The_American_Presi...,None,0112346,United States,http://dbpedia.org/resource/Rob_Reiner
2,18,113101,5.0,113101,Four Rooms,http://dbpedia.org/resource/Four_Rooms,None,0113101,United States,http://dbpedia.org/resource/Allison_Anders
3,18,113101,5.0,113101,Four Rooms,http://dbpedia.org/resource/Four_Rooms,None,0113101,United States,http://dbpedia.org/resource/Alexandre_Rockwell
4,18,113101,5.0,113101,Four Rooms,http://dbpedia.org/resource/Four_Rooms,None,0113101,United States,http://dbpedia.org/resource/Quentin_Tarantino
5,18,113101,5.0,113101,Four Rooms,http://dbpedia.org/resource/Four_Rooms,None,0113101,United States,http://dbpedia.org/resource/Robert_Rodriguez
6,22,112722,1710.0,112722,Copycat (film),http://dbpedia.org/resource/Copycat_(film),1995-10-27,0112722,United States,http://dbpedia.org/resource/Jon_Amiel
7,26,114057,16420.0,114057,Othello (1995 film),http://dbpedia.org/resource/Othello_(1995_film),1995-12-15,0114057,None,http://dbpedia.org/resource/Oliver_Parker
8,27,114011,9263.0,114011,Now and Then (film),http://dbpedia.org/resource/Now_and_Then_(film),None,0114011,United States,http://dbpedia.org/resource/Lesli_Linka_Glatter
9,28,114117,17015.0,114117,Persuasion (1995 film),http://dbpedia.org/resource/Persuasion_(1995_f...,None,0114117,United Kingdom,http://dbpedia.org/resource/Roger_Michell
